The goal of this notebook is to get the delay between the kinect stream and the other streams in the .xdf file. 

# The problem

Due to a bug in LSL_Kinect, the timestamps in the kinect streams are relative to the computer's startup time, whereas timestamps should be defined by LSL to enable synchronization with other streams.  
As a consequence, the kinect streams are delayed compared to the other streams, and the value of the delay is unknown (but constant).

# The solution
Hopefully, the kinect streams are also saved in .csv files, and the mouse streams are saved in other .csv files.

The solution is to use the .csv files corresponding to the .xdf file so to  :
- get the kinect-to-csv delay : the delay between the kinect stream and the corresponding csv file
- get the mouse-to-csv delay : the delay between mouse stream and the corresponding csv file  
- compute the kinect-to-mouse delay : the delay between the kinect stream and the mouse stream

Then, we can :
- compute the corrected kinect timestamps : the timestamps of the kinect stream shifted by the kinect-to-mouse delay



In [ ]:
# this should be the set to false for production use
# and set to true for testing (default) but can be already set to false from outside (e.g., by the test script)

if "doRunTests" not in globals():
    doRunTests = True

## Steps

1. Load the .xdf file
2. Load the kinect .csv file
3. Load the mouse .csv file
4. Compute the kinect-to-csv delay
5. Compute the mouse-to-csv delay
6. Compute the kinect-to-mouse delay



## Get the list of the needed csv files for the Reaching and Circle task

We get the .csv files corresponding to the .xdf file from `goodFiles.log` in the visit directory.

Only the Reaching and Circle tasks contain an xdf file with a kinect stream.

NOTE : for the mouse, we shall not use the data csv files, but only the marker csv files. For the kinect, we shall use the data and marker csv files (again a bug in LSL_Kinect).

The csv files that are expected are: 

- the kinect csv files (if they exist) 
    - `*_k.csv`: kinect data file 
    - `*_k_m.csv`: the kinect marker file

- the mouse csv marker files for the Reaching task (if they exist) 
    - `*_r_l_m_mau_np.csv`: mouse marker file for maximal arm use with the non-paretic arm
    - `*_r_l_m_mau_p.csv`: mouse marker file for maximal arm use with the paretic arm
    - `*_r_l_m_sau_np.csv`: mouse marker file for spontaneous arm use with the non-paretic arm
    - `*_r_l_m_sau_p.csv`: mouse marker file for spontaneous arm use with the paretic arm

- the mouse csv marker files for the Circle task (if they exist) 
    - `*_c_l_m_np.csv`: mouse marker file for the non-paretic arm
    - `*_c_l_m_p.csv`: mouse marker file for the paretic arm


The minimal set of files that are mandatory **for each task** are:
- the xdf file (the file that contains the kinect stream with wrong timestamps + the mouse stream with correct timestamps) 
- the kinect data csv file (the file that contains the kinect stream with correct timestamps)
- the kinect marker csv file (the file that contains the kinect marker stream with correct timestamps)
- one mouse marker csv file (the file that contains the mouse stream with correct timestamps)


In [ ]:
# we need the expected file list as created by the checkFilesInVisit notebook

# save current doRunTests value
doRunTestsOld = doRunTests

# run the outside notebooks (to get the functions)
doRunTests = False  # we need this to avoid running the tests in the outside notebooks
%run -i "checkFilesInVisit.ipynb"  # we need this to get the functions

# restore the doRunTests value
doRunTests = doRunTestsOld


In [ ]:
import os


def load_goodFiles(visit_path):
    """load the goodFiles.log file"""

    fullFname_goodFiles = os.path.join(visit_path, "goodFiles.log")
    goodFiles = []
    with open(fullFname_goodFiles, "r") as f:
        for line in f:
            goodFiles.append(line.strip())

    return goodFiles


def get_xdf_files(goodFiles):
    """get the xdf files from the goodFiles list"""
    # should be 2 xdf files: *r.xdf and *c.xdf
    xdf_files = []
    for f in goodFiles:
        if f.endswith(".xdf"):
            xdf_files.append(f)

    return xdf_files


def get_csv_files_for_xdf(xdf_file, goodFiles):
    """get the csv files that are associated with the xdf file"""
    csv_files = []
    xdf_fname_no_ext = os.path.splitext(xdf_file)[0]
    task_token = xdf_fname_no_ext.split("_")[-1]
    for f in goodFiles:
        if f.endswith(".csv"):
            if "_" + task_token + "_" in f:
                csv_files.append(f)

    return csv_files


def get_expected_file_endings(visit_path):
    """get the expected file endings for the visit using the ExpectedRearmVisit class"""
    expected_files = ExpectedRearmVisit(visit_path).expectedFiles
    expected_files = [
        x for x in expected_files if "Reaching" in x[0] or "Circle" in x[0]
    ]

    expected_endings = {"Reaching": [], "Circle": []}
    for expected_task_list in expected_files:
        if "Reaching" in expected_task_list[0]:
            for fname in expected_task_list[1]:
                if fname.endswith(".csv"):
                    expected_endings["Reaching"].append(fname.split("V?")[-1])
        if "Circle" in expected_task_list[0]:
            for fname in expected_task_list[1]:
                if fname.endswith(".csv"):
                    expected_endings["Circle"].append(fname.split("V?")[-1])

    return expected_endings


def check_expected_csv_files(xdf_csv_files):
    """Check if the expected csv files are present for the xdf files
    return a message with the errors found"""

    expected_endings = get_expected_file_endings(visit_path)
    msg = ""

    for xdf_csv in xdf_csv_files:
        xdf_fname = xdf_csv["xdf"]
        csv_files = xdf_csv["csv"]
        if xdf_fname.endswith("_r.xdf"):
            expected_ends = expected_endings["Reaching"]
        elif xdf_fname.endswith("_c.xdf"):
            expected_ends = expected_endings["Circle"]
        else:
            msg += f"Unexpected xdf file ending for {xdf_fname}\n"
            continue

        for ending in expected_ends:
            found = False
            for csv in csv_files:
                if csv.endswith(ending):
                    found = True
                    break
            if not found:
                msg += f"Expected file ending {ending} not found for {xdf_fname}\n"

    if xdf_csv_files is None or len(xdf_csv_files) == 0:
        msg = "No xdf files found\n"

    if len(xdf_csv_files) == 1:
        msg = f"Only one xdf file found (expecting two xdf files) \n"

    return msg


def get_xdf_and_corresponding_csv_files_in_file_list(goodFiles):
    """Get the xdf and corresponding csv files in the goodFiles list"""

    # check that gooFiles is a list of strings
    if not all(isinstance(x, str) for x in goodFiles):
        raise ValueError("goodFiles should be a list of strings")
    
    xdf_files = get_xdf_files(goodFiles)
    xdf_csv_files = []
    for f in xdf_files:
        csv_files = get_csv_files_for_xdf(f, goodFiles)
        xdf_csv_files.append({"xdf": f, "csv": csv_files})

    return xdf_csv_files


def check_mandatory_files(xdf_csv_files):
    """Check if the mandatory files are present for the xdf file"""

    import fnmatch

    mandatory_patterns = {
        "r": ["*_r_k.csv", "*_r_k_m.csv", "*_r_l*.csv"],
        "c": ["*_c_k.csv", "*_c_k_m.csv", "*_c_l*.csv"],
    }
    msg = ""

    for xdf_csv in xdf_csv_files:
        xdf_file = xdf_csv["xdf"]
        csv_files = xdf_csv["csv"]

        if xdf_file.endswith("_r.xdf"):
            mandatory_fname_pattern = mandatory_patterns["r"]
        elif xdf_file.endswith("_c.xdf"):
            mandatory_fname_pattern = mandatory_patterns["c"]
        else:
            msg += f"Unexpected xdf file ending for {xdf_file}\n"
            return msg

        for pattern in mandatory_fname_pattern:
            found = False
            for fname in csv_files:
                # if f.endswith(ending):
                if fnmatch.fnmatch(fname, pattern):
                    found = True
                    break
            if not found:
                msg += f"Mandatory file {pattern} not found for {xdf_file}\n"

    return msg

def get_xdf_and_corresponding_csv_files_in_visit(visit_path):
    """Get the xdf and corresponding csv files in the visit"""

    goodFiles = load_goodFiles(visit_path)
    xdf_csv_files = get_xdf_and_corresponding_csv_files_in_file_list(goodFiles)
    mandatory_files_error = check_mandatory_files(xdf_csv_files)

    return  xdf_csv_files, mandatory_files_error
     

if doRunTests:
    visit_path = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210419_V2"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210715_V3"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20210716_V1"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20210820_V2"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20211116_V3"

    xdf_csv_files, mandatory_files_error = get_xdf_and_corresponding_csv_files_in_visit(visit_path)

    if mandatory_files_error:
        print(
            "At least one mandatory file is missing: \n" + mandatory_files_error
        )

    print("XDF and CSV files that were found as expected:")
    for xdf_csv in xdf_csv_files:
        print("  " + xdf_csv["xdf"])
        for csv in xdf_csv["csv"]:
            print(f"    {csv}")

    print("\nReported error msg (you will see nothing below if all is as expected):")
    print(check_expected_csv_files(xdf_csv_files))

## Read a marker csv file

We have to read a marker csv file and return a list of [timestamp, marker] pairs.   
We want to read both kinect and mouse marker csv files, but the kinect markers lack the timestamp column.   


In [ ]:
import numpy as np
from datetime import datetime
import tempfile


def readMarkerCsv(fnameMarkerCsv):
    """Read a marker csv file and return a list of [timestamp, marker] pairs

    Parameters
    ----------
    fnameMarkerCsv : str
        Full filename of the marker csv file

    Returns
    -------

    out : list
        List of [timestamp, marker] pairs
            timestamp : float (in seconds)
            marker : str
    """

    if not fnameMarkerCsv.endswith(".csv"):
        raise ValueError("The file must be a csv file")

    with open(fnameMarkerCsv, "r") as fname:
        txt = fname.readlines()

    lines = [line.split(",") for line in txt]

    # PROBLEM: kinect markers lack the timestamp column
    # a marker is a line with 3 tokens : datetime,timestamp,marker \n
    # ACTION : add the timestamp column if we have only 2 columns
    for i in range(len(lines)):
        if len(lines[i]) == 2 and lines[i][0][0].isdigit():
            # create a timestamp that is in milliseconds (as in mouse markers)
            timestamp = (
                datetime.strptime(lines[i][0], "%Y-%m-%d %H:%M:%S.%f").timestamp()
                * 1000
            )
            lines[i].insert(1, str(timestamp))
            txt[i] = ",".join(lines[i])

    # PROBLEM: mouse markers lack the quotes around the multiline markers
    # find the lines where token 3 is "\n" = start of a multiline marker
    for i in range(len(lines)):
        line = lines[i]
        # find the start of a multiline marker
        if len(line) == 3 and line[2] == "\n":
            # add a " before the end of the line
            txt[i] = txt[i][:-1] + '"\n'
            # find the end of the multiline marker
            for j in range(i + 1, len(lines)):
                # if we have a normal one-line-marker
                if len(lines[j]) == 3:
                    # add a " before the end of the line (of the previous line)
                    txt[j - 1] = txt[j - 1][:-1] + '"\n'
                    break
                # if are at the end of the file
                if j == len(lines) - 1 and len(lines[j]) != 3:
                    # add a " before the end of the line
                    txt[j] = txt[j][:-1] + '"\n'
                    break

    # write the modified file to a temporary file and load it with np.loadtxt
    with tempfile.NamedTemporaryFile(mode="w", delete=False) as fname:
        fname.writelines(txt)
        tempFileName = fname.name
    lines = np.loadtxt(
        tempFileName, skiprows=3, delimiter=",", quotechar='"', dtype=str
    )

    # remove the first column (we shall need only the timestamp and the marker to compare with the xdf file)
    lines = np.delete(lines, 0, 1)

    # return a list of [timestamp, marker] pairs
    out = []
    for line in lines:
        # CAUTION : the timestamp must be in seconds (as in xdf files)
        timestamp = float(line[0]) / 1000
        marker = line[1]
        out.append([timestamp, marker])

    return out


if doRunTests:
    fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/Circle/ReArm_C1P02_20210322_1_c_l_m_np.csv"
    fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/Reaching/ReArm_C1P02_20210322_1_r_k_m.csv"
    fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1/ReArm_C1P02_20210306_V1_Circle/ReArm_C1P02_20210322_V1_c_k_m.csv"
    fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20210820_V2/ReArm_C1P07_20210820_V2_Reaching/ReArm_C1P07_20210820_V2_r_k_m.csv"

    visit_path = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210419_V2"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210715_V3"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20210716_V1"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20210820_V2"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20211116_V3"

    xdf_csv_files, mandatory_files_error = get_xdf_and_corresponding_csv_files_in_visit(visit_path)

    if not mandatory_files_error:
        # print the content of all marker csv files
        for xdf_csv in xdf_csv_files:
            for csv in xdf_csv["csv"]:
                if "_k_m" in csv or "_l_m" in csv:
                    print(f"\n----->>>>>>  Reading {csv}")
                    fnameMarkerCSV = os.path.join(visit_path, csv)
                    lines = readMarkerCsv(fnameMarkerCSV)
                    # print the lines
                    for line in lines:
                        print(line)

    else:
        print(
            "At least one mandatory file is missing: \n" + mandatory_files_error
        )
        

## Create a list of [timestamp, marker] pairs from all mouse csv files corresponding to one xdf file

A single list of [timestamp, marker] pairs from all mouse csv files corresponding to one xdf file should correspond to the mouse marker stream in the xdf file.

In [ ]:
def get_mouse_markers(xdf_csv):
    """Get the mouse markers from the csv files list and return them as a list of [timestamp, marker] pairs"""

    mouse_markers = []

    for csv in xdf_csv:
        if "_l_m" in csv:
            fnameMarkerCSV = os.path.join(visit_path, csv)
            lines = readMarkerCsv(fnameMarkerCSV)
            mouse_markers.extend(lines)

    # sort the markers by timestamp
    mouse_markers.sort(key=lambda x: x[0])

    return mouse_markers


if doRunTests:

    visit_path = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210306_V1"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210419_V2"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P02/ReArm_C1P02_20210715_V3"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20210716_V1"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20210820_V2"
    # visit_path = "../dat/ReArm.lnk/ReArm_C1P07/ReArm_C1P07_20211116_V3"

    xdf_csv_files, mandatory_files_error = get_xdf_and_corresponding_csv_files_in_visit(
        visit_path
    )

    mouse_markers_r = []
    mouse_markers_c = []

    if not mandatory_files_error:
        for xdf_csv in xdf_csv_files:
            if xdf_csv["xdf"].endswith("_r.xdf"):
                mouse_markers_r = get_mouse_markers(xdf_csv["csv"])
            if xdf_csv["xdf"].endswith("_c.xdf"):
                mouse_markers_c = get_mouse_markers(xdf_csv["csv"])
    else:
        print("At least one mandatory file is missing: \n" + mandatory_files_error)

    print(f"\nMouse markers reaching ({len(mouse_markers_r)} markers):")
    for marker in mouse_markers_r:
        print(marker)

    print(f"\nMouse markers circle ({len(mouse_markers_c)} markers):")
    for marker in mouse_markers_c:
        print(marker)